In [24]:
!pip install requests
!pip install fuzzywuzzy

In [25]:
import requests
import random
import json
from fuzzywuzzy import fuzz

In [26]:
#Step ONE
# Team Python Project- Recipe finder :)
#We use an input function within a defining function and formatted strings to determine users name and their bootcamp 
''' Ask user's information to welcome '''
def name():
    name = input("Please enter your name:  ")
    bootcamp = input("What is the name of your coding bootcamp?: ")
    print ("Hello {} and everyone joining today from {}".format(name.title(),bootcamp.title()))

In [27]:
''' Retrieve the outcome from API request '''
def recipe_finder(ingredient):   
    app_id = "9b99a79e"
    app_key = "685b85e54d9239c5ef342f4f65e84ae3"
    recipe_url = 'https://api.edamam.com/search?q={}&app_id={}&app_key={}'.format(ingredient, app_id, app_key)
    result = requests.get(recipe_url)
    stored_recipe = result.json()
    return stored_recipe['hits']

In [28]:
def ingredient_input():
    ''' Ask ingredient input from the user '''        
    ingredient = ''
    thinking_of_ingredient= input('Can you think of an ingredient? (y/n) ')
    
    ''' We use if statements to determine if users know what they are looking for
        if no - a random choice is generated
        if yes - users input the name of the ingredient to look up '''
    if thinking_of_ingredient == "n":
        print('It seems there are too many options, we will generate you a random one.')
        random_ingredients = ["cheese", "eggs", "chicken", "beef", "lamb"]
        ingredient = random.choice(random_ingredients)
        print("You are looking for:", ingredient)
    elif thinking_of_ingredient == "y":
        ingredient = input('Enter an ingredient you wish to use in a recipe: ')
        print("You are looking for:", ingredient)
    else:
        print("Invalid response, please try again")
        ingredient_input()
    
    return ingredient

In [29]:
def save_results(data_to_update: dict):
    ''' Save results to a file'''
    file_name = 'recipes.json'

    # if file exists, add new results to the file
    try:
        with open(file_name, 'r') as file:
            existing_data = json.load(file)
    except Exception as e:
        print(e)
        existing_data = []
        
    existing_data.insert(0, data_to_update) # add new data to the beginning of the file
    
    # Write the updated list back to the file
    with open(file_name, 'w') as file:
        json.dump(existing_data, file, indent=4)

In [30]:
''' Retrieve the label and URI from recipe's results'''
def recipe_results(ingredient):
    data_to_update = {}
    data_to_update['Ingredient'] = ingredient
    results_list = []
    
    # Step THREE: import requests for the recipe finder with the given ingredient
    results = recipe_finder(ingredient)
    # check if there are results for the input recipe
    if (len(results) >0):
        for result in results:
            result_dict = {}
            recipe = result['recipe']
            
            # store results to write to file
            result_dict['Label'] = recipe['label']
            result_dict['uri'] = recipe['uri']
            results_list.append(result_dict)
    else:
        print("It seems your ingredient cannot be found. Please try again...")
        ingredient = ingredient_input()
        recipe_results(ingredient)

    results_list = sorted(results_list, key=lambda item: list(item.values())[0]) # order the list results according to the label alphabetically
    data_to_update['Results'] = results_list
    save_results(data_to_update)

In [31]:
def pick_recipe():
    ''' Ask users which recipe they should choose'''
    file_name = 'recipes.json'

    # if file exists, get data from the file
    try:
        with open(file_name, 'r') as file:
            recipe_results = json.load(file)
    
        preferred_recipe = input("Which recipe you want to choose? ")
        
        for recipe_result in recipe_results:
            for result in recipe_result["Results"]:
                recipe_name = result['Label']
                if fuzz.partial_ratio(recipe_name.lower(), preferred_recipe.lower()) >=70: # find match results between user input and stored recipes
                    print("Label: {}".format(recipe_name))
                    print("URI: {}".format(result['uri']))
    except Exception:
        print('Error opening the result file.')


In [32]:
def ask_to_choose_recipe():
    chosen_recipe = input("Oh right, do you want to choose the recipe from ingredien(s) you just searched? (y/n) ")
    if chosen_recipe.lower() == 'y':
        pick_recipe()
    elif chosen_recipe.lower() == 'n':
        print("Hope you have found what you need. See you next time!")
    else:
        print("Invalid response, please try again...")
        ask_to_choose_recipe()

In [33]:
def continue_search():
    ''' Allow users to continue another search '''
    is_continue = True
    while (is_continue == True):
        continue_request = input("Do you want to continue searching an ingredient? (y/n) ")
        if (continue_request.lower() == 'y'):
            is_continue = True
            ingredient = ingredient_input()   
            recipe_results(ingredient)
        elif (continue_request.lower() == 'n'):
            # Step SIX: check if the users want to choose 
            is_continue = False
            ask_to_choose_recipe()                
        else:
            is_continue = True
            print("Invalid response, please try again...")

In [34]:
def search():
    make_a_recipe = input("Do you want to make a meal? (y/n) ")
    if make_a_recipe.lower() == "y":
        print("Start thinking about which ingredient you wish to include ...")
        
        # Step TWO: we use if statements to determine if users know what they are looking for  
        ingredient = ingredient_input()   
        
        # Step FOUR: retrieve the recipe results and write to a JSON file
        recipe_results(ingredient)
        
        #Step FIVE: we use a while loop and booleans to determine if you want to search for another recipe or not 
        continue_search()
        
    elif make_a_recipe.lower() == "n":
            print("Hope you have found what you need. See you next time!")
    else:
        print("Invalid response, please try again")
        search()
    

In [35]:
if __name__ == '__main__':
    name()
    search()

Hello Linh and everyone joining today from Fbdfb
Start thinking about which ingredient you wish to include ...
You are looking for: egg
Label: Fried Egg
URI: http://www.edamam.com/ontologies/edamam.owl#recipe_3bf20fd757bd45a89ed149db85d081b7
Label: Fried Egg
URI: http://www.edamam.com/ontologies/edamam.owl#recipe_0eb708f983f08541458392788dff059b
